## Observations and Insights 

In [282]:
%matplotlib notebook

In [283]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_rpt_df = pd.merge(mouse_metadata,study_results,how='outer', on='Mouse ID')

# Display the data table for preview
mouse_rpt_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [284]:
# Checking the number of mice.
num_mice = mouse_rpt_df.drop_duplicates('Mouse ID')['Mouse ID'].count()
print("Total number of mice in study: " + str(num_mice))

Total number of mice in study: 249


In [285]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_rpt_df['Duplicate'] = mouse_rpt_df.duplicated(['Mouse ID','Timepoint'])
dup_mouse = mouse_rpt_df[mouse_rpt_df['Duplicate']
                          ==True].drop_duplicates(['Mouse ID'])

dup_mouseid = dup_mouse['Mouse ID'].tolist()

print("Mouse ID: " + str(dup_mouseid))
print("Shows up as a duplicate for Mouse ID and Timepoint")

Mouse ID: ['g989']
Shows up as a duplicate for Mouse ID and Timepoint


In [286]:
# Optional: Get all the data for the duplicate mouse ID. 
lsdup_mouse = mouse_rpt_df[mouse_rpt_df['Duplicate']==True]
lsdup_mouse

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,Duplicate
909,g989,Propriva,Female,21,26,0,45.000000,0,True
911,g989,Propriva,Female,21,26,5,47.570392,0,True
913,g989,Propriva,Female,21,26,10,49.880528,0,True
915,g989,Propriva,Female,21,26,15,53.442020,0,True
917,g989,Propriva,Female,21,26,20,54.657650,1,True


In [287]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
mousend_rpt_df = mouse_rpt_df.set_index('Mouse ID').drop(labels=dup_mouseid)
mousend_rpt_df

,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,Duplicate
Mouse ID,,,,,,,,
k403,Ramicane,Male,21,16,0,45.000000,0,False
k403,Ramicane,Male,21,16,5,38.825898,0,False
k403,Ramicane,Male,21,16,10,35.014271,1,False
k403,Ramicane,Male,21,16,15,34.223992,1,False
k403,Ramicane,Male,21,16,20,32.997729,1,False
...,...,...,...,...,...,...,...,...
z969,Naftisol,Male,9,30,25,63.145652,2,False
z969,Naftisol,Male,9,30,30,65.841013,3,False
z969,Naftisol,Male,9,30,35,69.176246,4,False


In [288]:
# Checking the number of mice in the clean DataFrame.
cmousend_rpt_df = mousend_rpt_df.reset_index()
cmousend_rpt_df = cmousend_rpt_df.drop(['Duplicate'], axis=1)
num_micend = cmousend_rpt_df.drop_duplicates('Mouse ID')['Mouse ID'].count()
print("Total number of mice in clean DataFrame: " + str(num_micend))

Total number of mice in clean DataFrame: 248


## Summary Statistics

In [289]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
gcmousend_rpt = cmousend_rpt_df.groupby(['Drug Regimen'])

gcmousend_rpt_df = pd.DataFrame({
    'Mean' : gcmousend_rpt['Tumor Volume (mm3)'].mean(),
    'Median' : gcmousend_rpt['Tumor Volume (mm3)'].median(),
    'Variance' : gcmousend_rpt['Tumor Volume (mm3)'].var(),
    'Standard Deviation' : gcmousend_rpt['Tumor Volume (mm3)'].std(),
    'SEM' : gcmousend_rpt['Tumor Volume (mm3)'].sem()
})

# This method is the most straighforward, creating multiple series and putting them all together at the end.
gcmousend_rpt_df.style.format({'Mean':'{:.2f}','Median':'{:.2f}','Variance':'{:.2f}',
                               'Standard Deviation':'{:.2f}','SEM':'{:.2f}'})

,Mean,Median,Variance,Standard Deviation,SEM
Drug Regimen,,,,,
Capomulin,40.68,41.56,24.95,4.99,0.33
Ceftamin,52.59,51.78,39.29,6.27,0.47
Infubinol,52.88,51.82,43.13,6.57,0.49
Ketapril,55.24,53.70,68.55,8.28,0.60
Naftisol,54.33,52.51,66.17,8.13,0.60
Placebo,54.03,52.29,61.17,7.82,0.58
Propriva,52.32,50.45,43.85,6.62,0.54
Ramicane,40.22,40.67,23.49,4.85,0.32
Stelasyn,54.23,52.43,59.45,7.71,0.57


In [290]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [291]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [292]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [293]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [294]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [295]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [296]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [297]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [298]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [299]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [300]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
